In [ ]:
!pip install pyaztro
import pyaztro
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
df = pd.read_csv('/content/1970-2021_DISASTERS.xlsx - emdat data.csv')
df = df.fillna(0)
kolom_dihapus = ['No Injured', 'No Affected', 'No Homeless', 'Total Affected',
                 'Adm Level', 'Admin1 Code', 'Admin2 Code', 'Geo Locations',
                 'River Basin', 'Dis Mag Value', 'Aid Contribution', 'Declaration',
                 'Appeal', 'OFDA Response', 'Associated Dis2', 'Disaster Subsubtype',
                 'Disaster Subtype','Disaster Subgroup', 'Glide','Event Name','ISO',
                 'Location','Associated Dis','Origin','Dis Mag Scale','Latitude',
                 'Longitude','Local Time','Dis No']

df = df.drop(columns=kolom_dihapus)

In [ ]:
grouped_df = df.groupby('Disaster Type')

def on_button_clicked(b):
    with output:
        clear_output()
        global selected_rows_disaster
        disaster_type = b.description
        selected_rows_disaster = grouped_df.get_group(disaster_type)
        print(f'Disaster Type: {disaster_type}\n')

button_list = []
for disaster_type in df['Disaster Type'].unique():
    button = widgets.Button(description=disaster_type)
    button.on_click(on_button_clicked)
    button_list.append(button)

output = widgets.Output()
display(widgets.VBox(button_list + [output]))

min_year = int(df['Year'].min())
max_year = int(df['Year'].max())
years_range_slider = widgets.IntRangeSlider(
    description='Select years:',
    min=min_year,
    max=max_year,
    step=1,
    value=(min_year, max_year)
)

def on_years_range_change(change):
    selected_rows_years = df[(df['Year'] >= change.new[0]) & (df['Year'] <= change.new[1])]
    with output:
        clear_output()
        print(f'Selected Years: {change.new[0]} - {change.new[1]}\n')
        
        country_with_max_deaths = selected_rows_years.groupby('Country').sum().nlargest(1, 'Total Deaths')
        if not country_with_max_deaths.empty:
            country_data = selected_rows_disaster[selected_rows_disaster['Country'] == country_with_max_deaths.index[0]]
            plt.plot(country_data['Year'], country_data['Total Deaths'])
            plt.xlabel('Year')
            plt.ylabel('Total Deaths')
            plt.title(f"Total Deaths from {selected_rows_disaster['Disaster Type'].unique()[0]} in {country_with_max_deaths.index[0]}")
            plt.show()

years_range_slider.observe(on_years_range_change, names='value')

display(years_range_slider)

IntRangeSlider(value=(1970, 2021), description='Select years:', max=2021, min=1970)